In [1]:
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    text = file.read()
print(text)

Ag cr REF ELEMENT T 6/12AG 1.   0.   0.   0.S   200.000  1235.080 1000.        1
 2.07216824E+00 2.46393729E-03-1.34351116E-06 3.69321107E-10 0.00000000E+00    2
-6.37725170E+02-7.18810718E+00 2.25225065E+00 5.43263008E-03-1.32153990E-05    3
 1.50423505E-08-5.94991675E-12-8.23132027E+02-8.86835190E+00 0.00000000E+00    4
Ag liq REF ELEMENTT 6/12AG 1.   0.   0.   0.L  1235.080  6000.000 1235.08      1
 4.04091552E+00-3.49297186E-05 1.60169701E-08-2.96225835E-12 1.92332513E-16    2
-4.77718035E+02-1.78491707E+01 0.00000000E+00 0.00000000E+00 0.00000000E+00    3
 0.00000000E+00 0.00000000E+00 0.00000000E+00 0.00000000E+00 0.00000000E+00    4
Ag                g10/97AG 1.   0.   0.   0.G   200.000  6000.000 1000.        1
 2.49722745E+00-5.96378605E-07 7.16732589E-09-4.57774728E-12 7.79345485E-16    2
 3.35216939E+04 6.58031521E+00 2.50000000E+00 0.00000000E+00 0.00000000E+00    3
 0.00000000E+00 0.00000000E+00 3.35200237E+04 6.56281935E+00 3.42653987E+04    4
Ag+               g10/97AG 1

Задача: распарсить файл по принципу:

`{"Ag" : [2.07216824E+00, 2.46393729E-03, -1.34351116E-06, 3.69321107E-10, 0.00000000E+00, -6.37725170E+02, -7.18810718E+00, 2.25225065E+00, 5.43263008E-03, -1.32153990E-05, 1.50423505E-08, -5.94991675E-12, -8.23132027E+02, -8.86835190E+00, 0.00000000E+00]}`

In [2]:
import json

In [3]:
def read_four_lines(f):
    first = f.readline()
    second = f.readline()
    third = f.readline()
    fourth = f.readline()
    return (first, second, third, fourth)

In [4]:
def parse_quadruple(q):
    element = q[0][0:q[0].index(' ')]
    coefs = []
    stride = 15
    for i in range(1, 4):
        for j in range(0, 5):
            raw_data = q[i][j*stride:(j+1)*stride]
            try:
                data = float(raw_data)
            except ValueError:
                data = float(0)
            coefs += [data]
    element_data = {element : coefs}
    return element_data

In [5]:
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    q = read_four_lines(file)
    print(parse_quadruple(q))

{'Ag': [2.07216824, 0.00246393729, -1.34351116e-06, 3.69321107e-10, 0.0, -637.72517, -7.18810718, 2.25225065, 0.00543263008, -1.3215399e-05, 1.50423505e-08, -5.94991675e-12, -823.132027, -8.8683519, 0.0]}


In [6]:
result = {}
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    for i in range(0, 2972): #Я вручную посчитал сколько там должно быть элементов
        q = read_four_lines(file)
        buf = parse_quadruple(q)
        result.update(buf)
parsed_to_json = json.dumps(result, indent=4)

In [7]:
print(result)

{'Ag': [2.49722745, -5.96378605e-07, 7.16732589e-09, -4.57774728e-12, 7.79345485e-16, 33521.6939, 6.58031521, 2.5, 0.0, 0.0, 0.0, 0.0, 33520.0237, 6.56281935, 34265.3987], 'Ag+': [9.72687035, -0.0040147218, 7.47796464e-07, -1.76595533e-11, -4.14279861e-15, 117714.726, -39.1847888, 0.167072904, 0.0230034783, -8.05675031e-05, 1.14647031e-07, -5.08119362e-11, 122365.909, 14.9717871, 122928.919], 'Ag-': [2.50067694, 0.0, 0.0, 0.0, 0.0, 17665.4697, 5.86586664, 2.50067694, 0.0, 0.0, 0.0, 0.0, 17665.4697, 5.86586664, 18411.0465], 'AIR': [3.08792717, 0.00124597184, -4.23718945e-07, 6.74774789e-11, -3.97076972e-15, -995.262755, 5.9596093, 3.5683962, -0.000678729429, 1.55371476e-06, -3.2993706e-12, -4.66395387e-13, -1062.34659, 3.71582965, -15.0965], 'AL(cr)': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.01032822, 0.012077564, -2.62098556e-05, 2.64295083e-08, -9.01946533e-12, -654.447545, -5.00441242, 0.0], 'AL(L)': [3.83089866, -2.09027129e-05, 1.04271684e-08, -2.04841051e-12, 1.39565517e-16, -99.7961

In [8]:
with open("parsed.json", "w", encoding="UTF-8") as file:
    file.write(parsed_to_json)